In [2]:
import pandas as pd
from pathlib import Path

[PosixPath('../axolotl24_shared_task/data/german/axolotl.test.surprise.gold.tsv'),
 PosixPath('../axolotl24_shared_task/data/russian/axolotl.test.ru.gold.tsv'),
 PosixPath('../axolotl24_shared_task/data/finnish/axolotl.test.fi.gold.tsv')]

In [94]:
dft = []
for path in list(Path('../axolotl24_shared_task').glob('**/*gold*tsv')):
    print('*** Subset: ', p)
    df = pd.read_csv(path, sep='\t')

    #For good ARI for a word WSD is enough when 1) there are no gained senses, or 2) there is 1 old sense and 1 new sense

    dfs = pd.concat([df[df.period==p].groupby('word').sense_id.unique().apply(set).to_frame(p) 
               for p in ('new','old')], axis=1)
    for p in ('new','old'):
        f = 'n'+p
        dfs[f] = dfs[p].apply(len)
        print(f+':\n', dfs[f].value_counts(normalize=True).head(5), sep='')

    print('Nulls:',dfs.isnull().sum())

    ngained = (dfs.new - dfs.old).apply(len)
    props = ngained.value_counts(normalize=True)
    print('Proportions of words with N gained senses:\n',props, sep='')


    x1 = ((dfs.nold==1)&(dfs.nnew==1)).mean()
    x2 = ((ngained==0) | dfs.eval("nold==1 & nnew==1")).mean()
    
    dft.append(pd.DataFrame({
        'words with 1 new sense:': (dfs.nnew==1).mean(),
        'words with 1 new and 1 old sense:': x1,
        'words with no gained senses': props.loc[0] if 0 in props.index else 0,
        'words with 1 new and 1 old sense, or no gained senses': x2,
    }, index=[path.name]))


*** Subset:  old
nnew:
2    0.583333
4    0.166667
1    0.125000
6    0.041667
5    0.041667
Name: nnew, dtype: float64
nold:
2    0.541667
1    0.166667
3    0.166667
4    0.083333
6    0.041667
Name: nold, dtype: float64
Nulls: new     0
old     0
nnew    0
nold    0
dtype: int64
Proportions of words with N gained senses:
0    0.666667
1    0.208333
5    0.041667
3    0.041667
2    0.041667
dtype: float64
*** Subset:  old
nnew:
3    0.436019
4    0.241706
2    0.113744
5    0.085308
6    0.052133
Name: nnew, dtype: float64
nold:
1    0.554502
2    0.222749
3    0.109005
5    0.033175
4    0.028436
Name: nold, dtype: float64
Nulls: new     0
old     0
nnew    0
nold    0
dtype: int64
Proportions of words with N gained senses:
2     0.559242
3     0.274882
5     0.056872
4     0.037915
1     0.033175
6     0.028436
8     0.004739
11    0.004739
dtype: float64
*** Subset:  old
nnew:
1    0.596364
2    0.156364
3    0.087273
4    0.061818
5    0.029091
Name: nnew, dtype: float64
nold:
1 

In [95]:
pd.concat(dft).T

,axolotl.test.surprise.gold.tsv,axolotl.test.ru.gold.tsv,axolotl.test.fi.gold.tsv
words with 1 new sense:,0.125000,0.004739,0.596364
words with 1 new and 1 old sense:,0.041667,0.004739,0.501818
words with no gained senses,0.666667,0.000000,0.741818
"words with 1 new and 1 old sense, or no gained senses",0.708333,0.004739,0.781818
